# Using prompto with Vertex AI

In [1]:
from prompto.settings import Settings
from prompto.experiment import Experiment
from dotenv import load_dotenv
import warnings
import os

When using `prompto` to query models from the Vertex AI API, lines in our experiment `.jsonl` files must have `"api": "vertexai"` in the prompt dict. 

## Environment variables

For the [Vertex AI API](https://alan-turing-institute.github.io/prompto/docs/vertexai/), there are two environment variables that could be set:
- `VERTEXAI_PROJECT_ID`: the project-id for the Vertex AI API
- `VERTEXAI_LOCATION_ID`: the location-id for the Vertex AI API

As mentioned in the [environment variables docs](https://alan-turing-institute.github.io/prompto/docs/environment_variables/#model-specific-environment-variables), there are also model-specific environment variables too which can be utilised. In particular, when you specify a `model_name` key in a prompt dict, one could also specify a `VERTEXAI_PROJECT_ID_model_name` or `VERTEXAI_LOCATION_ID_model_name` environment variables to indicate the project-id and location-id for that particular model (where "model_name" is replaced to whatever the corresponding value of the `model_name` key is).

To set environment variables, one can simply have these in a `.env` file which specifies these environment variables as key-value pairs:
```
VERTEXAI_PROJECT_ID=<YOUR-VERTEXAI-PROJECT-ID>
VERTEXAI_LOCATION_ID=<YOUR-VERTEXAI-LOCATION-ID>
```

If you make this file, you can run the following which should return `True` if it's found one, or `False` otherwise:

In [2]:
load_dotenv(dotenv_path=".env")

True

Now, we obtain those values. We some warnings if the `VERTEXAI_PROJECT_ID` or `VERTEXAI_LOCATION_ID` environment variables haven't been set. However, note that when using Vertex AI, you can actually set the default project and location using the `gcloud` CLI, so these aren't strictly necessary.

In [3]:
VERTEXAI_PROJECT_ID = os.environ.get("VERTEXAI_PROJECT_ID")
if VERTEXAI_PROJECT_ID is None:
    warnings.warn("VERTEXAI_PROJECT_ID is not set")

In [4]:
VERTEXAI_LOCATION_ID = os.environ.get("VERTEXAI_LOCATION_ID")
if VERTEXAI_LOCATION_ID is None:
    warnings.warn("VERTEXAI_LOCATION_ID is not set")

## Types of prompts

With the Vertex AI API, the prompt (given via the `"prompt"` key in the prompt dict) can take several forms:
- a string: a single prompt to obtain a response for
- a list of strings: a sequence of prompts to send to the model
    - this is useful in the use case of simulating a conversation with the model by defining the user prompts sequentially
- a list of dictionaries with keys "role" and "parts", where "role" is one of "user", "model", or "system" and "parts" is the message/prompt to the model
    - this is useful in the case of passing in some conversation history or to pass in a system prompt to the model
    - note that only one prompt in the list can be a system prompt and it must be the first - the rest must be user or model prompts

The last format is also useful for the case where you want to pass in some conversation history to the model. It is also how we can define multimodal prompts to the model - more details in the [Multimodal prompting with Vertex AI notebook](./vertexai-multimodal.ipynb).

We have created an input file in [data/input/vertexai-example.jsonl](https://github.com/alan-turing-institute/prompto/blob/main/examples/vertexai/data/input/vertexai-example.jsonl) with an example of each of these cases as an illustration.

In [5]:
settings = Settings(data_folder="./data", max_queries=30)
experiment = Experiment(file_name="vertexai-example.jsonl", settings=settings)

We set `max_queries` to 30 so we send 30 queries a minute (every 2 seconds).

In [6]:
print(settings)

Settings: data_folder=./data, max_queries=30, max_attempts=3, parallel=False
Subfolders: input_folder=./data/input, output_folder=./data/output, media_folder=./data/media


In [7]:
len(experiment.experiment_prompts)

6

We can see the prompts that we have in the `experiment_prompts` attribute:

In [8]:
experiment.experiment_prompts

[{'id': 0,
  'api': 'vertexai',
  'model_name': 'gemini-1.5-flash-002',
  'prompt': 'How does technology impact us?',
  'parameters': {'candidate_count': 1,
   'temperature': 1,
   'max_output_tokens': 1000}},
 {'id': 1,
  'api': 'vertexai',
  'model_name': 'gemini-1.0-pro-002',
  'prompt': 'How does technology impact us?',
  'parameters': {'candidate_count': 1,
   'temperature': 1,
   'max_output_tokens': 1000}},
 {'id': 2,
  'api': 'vertexai',
  'model_name': 'gemini-1.5-flash-002',
  'prompt': ['How does international trade create jobs?',
   'I want a joke about that'],
  'parameters': {'candidate_count': 1,
   'temperature': 1,
   'max_output_tokens': 1000}},
 {'id': 3,
  'api': 'vertexai',
  'model_name': 'gemini-1.5-flash-002',
  'prompt': [{'role': 'system',
    'parts': 'You are a helpful assistant designed to answer questions briefly.'},
   {'role': 'user',
    'parts': 'What efforts are being made to keep the hakka language alive?'}],
  'parameters': {'candidate_count': 1,
  

- In the first prompt (`"id": 0`), we have a `"prompt"` key which is a string and specify a `"model_name"` key to be "gemini-1.5-flash"
- In the second prompt (`"id": 1`), we have a `"prompt"` key is also a string but we specify a `"model_name"` key to be "gemini-1.0-pro".
- In the third prompt (`"id": 2`), we have a `"prompt"` key which is a list of strings.
- In the fourth prompt (`"id": 3`), we have a `"prompt"` key which is a list of dictionaries. These dictionaries have a "role" and "parts" key. This acts as passing in a system prompt. Here, we just have a system prompt before a user prompt.
- In the fifth prompt (`"id": 4`), we have a `"prompt"` key which is a list of dictionaries. These dictionaries have a "role" and "parts" key but here, we have a user prompt and then a system prompt. As mentioned above, only the first prompt in the list can be a system prompt. We should get an error for this particular prompt.
- In the sixth prompt (`"id": 5`), we have a `"prompt"` key which is a list of dictionaries. These dictionaries have a "role" and "parts" key. Here, we have a system prompt and a series of user/model interactions before finally having a user prompt. This acts as passing in a system prompt and conversation history.

Note that for each of these prompt dicts, we have `"model_name": "gemini-1.5-flash"`, besides `"id": 1` where we have `"model_name": "gemini-1.0-pro"`.

## Safety filters with VertexAI API

With the Gemini API, it is possible to configure the safety filters (see the [safety settings docs](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes)). We can set the `"safety_filter"` key in the prompt dict where the options are:
- `"none"`: corresponds to "Block none" or `BLOCK_NONE`
- `"few"`: corresponds to "Block few" or `BLOCK_ONLY_HIGH`
- `"default"` or `"some"`: corresponds to "Block some" or `BLOCK_HIGH_AND_MEDIUM`
- `"most"`: corresponds to "Block most" or `BLOCK_LOW_AND_ABOVE`

In the example input file, we have set the `"safety_filter"` key to each of these options.

## Running the experiment

We now can run the experiment using the async method `process` which will process the prompts in the input file asynchronously. Note that a new folder named `timestamp-vertexai-example` (where "timestamp" is replaced with the actual date and time of processing) will be created in the output directory and we will move the input file to the output directory. As the responses come in, they will be written to the output file and there are logs that will be printed to the console as well as being written to a log file in the output directory.

In [9]:
responses, avg_query_processing_time = await experiment.process()

Sending 6 queries at 30 QPM with RI of 2.0s (attempt 1/3): 100%|██████████| 6/6 [00:14<00:00,  2.48s/query]
Waiting for responses (attempt 1/3): 100%|██████████| 6/6 [00:00<00:00,  7.46query/s]


We can see that the responses are written to the output file, and we can also see them as the returned object. From running the experiment, we obtain prompt dicts where there is now a `"response"` key which contains the response(s) from the model.

For the case where the prompt is a list of strings, we see that the response is a list of strings where each string is the response to the corresponding prompt.

In [10]:
responses

[{'id': 0,
  'api': 'vertexai',
  'model_name': 'gemini-1.5-flash-002',
  'prompt': 'How does technology impact us?',
  'parameters': {'candidate_count': 1,
   'temperature': 1,
   'max_output_tokens': 1000},
  'timestamp_sent': '18-10-2024-10-58-24',
  'response': "Technology's impact on us is profound and multifaceted, affecting nearly every aspect of our lives.  Here's a breakdown of some key areas:\n\n**Positive Impacts:**\n\n* **Improved Communication:**  Instant communication across vast distances through email, messaging apps, video calls, and social media connects people globally.\n* **Increased Efficiency and Productivity:** Automation, software, and tools streamline tasks in various fields, boosting productivity in work and daily life.\n* **Access to Information:** The internet provides unparalleled access to information, education, and diverse perspectives, empowering individuals and fostering learning.\n* **Advancements in Healthcare:** Medical technology leads to earlier d

Also notice how with the Vertex AI API, we record some additional information related to the safety attributes.

## Running the experiment via the command line

We can also run the experiment via the command line. The command is as follows (assuming that your working directory is the current directory of this notebook, i.e. `examples/vertexai`):
```bash
prompto_run_experiment --file data/input/vertexai-example.jsonl --max-queries 30
```